# Feature extraction in MASS
some critical features

200 Hz

broadband [0,40]

In [17]:
#external libraries
import os
import dotenv
import pandas as pd
import numpy as np
from scipy import signal as sg
import pickle as pkl
import time
import matplotlib.pyplot as plt

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']

In [18]:
#define features path
masspath=datapath+"/MASS"
featurespath=masspath+"/features"
        
windowDurations=[0.5,1,1.5,2]

## Load data

In [19]:
#load data THIS NEEDS REFINEMENT AND CONVERGENCE TO USE WITH MULTIPLE DATABASES
def loadMASSSpindles(path):
    #signalsMetadata
    signalsMetadata=pd.read_csv(path+'\\signals\\signalsMetadata.csv')
    signalsMetadata['subjectId']=signalsMetadata.apply(
        lambda row: str(row.subjectId).zfill(4),axis=1)

    #load signals from pickle
    signals={}
    for index, row in signalsMetadata.iterrows():
        signalpath=path+"/signals/"+row.file
        cFile = open(signalpath, 'rb')
        signals[row.subjectId]= pkl.load(cFile)
        cFile.close()

    #spindle annotations
    annotations=pd.read_csv(path+'\\annotations\\annotations.csv')
    annotations['subjectId']=annotations.apply(
        lambda row: str(row.subjectId).zfill(4),axis=1)
    annotations['labelerId']=annotations.apply(
        lambda row: str(row.labelerId).zfill(4),axis=1)
    
    #add stop and index colums
    annotations=annotations.merge(signalsMetadata[['subjectId','samplerate']],how='left',on='subjectId')
    annotations['stopTime']=annotations.apply(
        lambda row: row.startTime+row.duration , axis=1)
    annotations['startInd']=annotations.apply(
        lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
    annotations['stopInd']=annotations.apply(
        lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

    
    return signals, annotations, signalsMetadata
    

In [20]:
signals, annotations, signalsMetadata = loadMASSSpindles(masspath)

In [21]:
signalsMetadata.head(5)

,subjectId,file,channel,duration,samplerate
0,0001,MASS_0001.pkl,C3-CLE,28956.0,256
1,0002,MASS_0002.pkl,C3-CLE,35016.0,256
2,0003,MASS_0003.pkl,C3-CLE,36760.0,256
3,0004,MASS_0004.pkl,C3-CLE,28004.0,256
4,0005,MASS_0005.pkl,C3-CLE,31244.0,256


## Resample to 200 Hz
it is important to make it this way, original data should never be modifyed

In [22]:
samplerate=200

In [23]:
print(256*25/32)   #<- TBD: make this automatic
# WARNING: parameters hardcoded ----------------------->
myUp=25
myDown=32
#<------------------------------------------------------

200.0


In [24]:
signalsMetadata

,subjectId,file,channel,duration,samplerate
0,0001,MASS_0001.pkl,C3-CLE,28956.0,256
1,0002,MASS_0002.pkl,C3-CLE,35016.0,256
2,0003,MASS_0003.pkl,C3-CLE,36760.0,256
3,0004,MASS_0004.pkl,C3-CLE,28004.0,256
4,0005,MASS_0005.pkl,C3-CLE,31244.0,256
5,0006,MASS_0006.pkl,C3-CLE,28990.0,256
6,0007,MASS_0007.pkl,C3-CLE,28302.0,256
7,0008,MASS_0008.pkl,C3-CLE,26846.0,256
8,0009,MASS_0009.pkl,C3-CLE,29834.0,256
9,0010,MASS_0010.pkl,C3-CLE,25930.0,256


In [25]:
#Uncomment for graphical representation
""" for ind, row in signalsMetadata.iterrows():
    original=signals[row.subjectId]
    aux=sg.resample_poly(original,up=myUp,down=myDown)
    break
originalTime=np.arange(len(original))/256
auxTime=np.arange(len(aux))/200
timeStart=10**4
timeEnd=1.2*10**4
roi1=(originalTime>timeStart)&(originalTime<timeEnd)
roi2=(auxTime>timeStart)&(auxTime<timeEnd)
fig=px.line(x=originalTime[roi1],y=original[roi1])
fig.add_scatter(x=auxTime[roi2],y=aux[roi2],name="resampled")
fig.write_html("resampling.html") """

' for ind, row in signalsMetadata.iterrows():\n    original=signals[row.subjectId]\n    aux=sg.resample_poly(original,up=myUp,down=myDown)\n    break\noriginalTime=np.arange(len(original))/256\nauxTime=np.arange(len(aux))/200\ntimeStart=10**4\ntimeEnd=1.2*10**4\nroi1=(originalTime>timeStart)&(originalTime<timeEnd)\nroi2=(auxTime>timeStart)&(auxTime<timeEnd)\nfig=px.line(x=originalTime[roi1],y=original[roi1])\nfig.add_scatter(x=auxTime[roi2],y=aux[roi2],name="resampled")\nfig.write_html("resampling.html") '

In [26]:
#1. resample
for ind, row in signalsMetadata.iterrows():
    signals[row.subjectId]=sg.resample_poly(signals[row.subjectId],up=myUp,down=myDown)

#2. update metadata
signalsMetadata["samplerate"]=samplerate
signalsMetadata["duration"]=signalsMetadata.apply(
    lambda row: len(signals[row.subjectId])/row.samplerate,
    axis=1) #it should be the exact same duration

#3. update annotations
annotations['samplerate']=samplerate
annotations['startInd']=annotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate),
    axis=1)
annotations['stopInd']=annotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate),
    axis=1)


In [27]:
signalsMetadata

,subjectId,file,channel,duration,samplerate
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200
5,0006,MASS_0006.pkl,C3-CLE,28990.0,200
6,0007,MASS_0007.pkl,C3-CLE,28302.0,200
7,0008,MASS_0008.pkl,C3-CLE,26846.0,200
8,0009,MASS_0009.pkl,C3-CLE,29834.0,200
9,0010,MASS_0010.pkl,C3-CLE,25930.0,200


## Preprocess

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [28]:
def preprocessVector(vector,samplerate):
    
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Robust Z-score
    from sklearn.preprocessing import RobustScaler
    vector_preprocessed=RobustScaler().fit_transform(vector.reshape(-1, 1) )
    return vector_preprocessed. flatten()

In [29]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## Extract features

### folder estructure

In [30]:
fspath=featurespath+'/'+str(samplerate)+'fs/'
if not os.path.isdir(fspath):
    os.mkdir(fspath)
for window in windowDurations:
    windowPath=fspath+str(window)+'win'
    if not os.path.isdir(windowPath):
        os.mkdir(windowPath)
    for subject in signalsMetadata.subjectId:
        subjectPath=windowPath+'/'+subject
        if not os.path.isdir(subjectPath):
            os.mkdir(subjectPath)

### band definition

In [31]:
bands={
    'delta1':[0.1,2],
    'delta2':[2,4],
    'theta':[4,8],
    'alpha':[8,13],
    'sigma':[11,16],
    'beta1':[13,19],
    'beta2':[19,30]
    }

### computation

In [32]:
for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        signal=signals[subject]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        
        #6. Hjort Activity
        characteristic='hjortActivity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortActivity(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortActivity(filtered,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #7. Hjort Mobility
        characteristic='hjortMobility'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortMobility(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortMobility(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #8. Hjort Complexity
        characteristic='hjortComplexity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortComplexity(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortComplexity(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #9. PetrosianFractalDimension
        characteristic='petrosian'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=petrosianFractalDimension(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=petrosianFractalDimension(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=sigmaindex(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
            

saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_broadband.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta1.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta2.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_theta.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_alpha.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_sigma.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_beta1.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_beta2.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortMobility_broadband.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortMobility_delta1.fd
saving feature: D:/SpinCo/MASS/features/200fs/0.5win/0001/0.5_0001_hjortMobility_delta2.f